# Horn Radio Telescope Convert files from .ast, .hot to Comma Separated Values (.csv)

### Glen Langston, 2021 October 13

This notebook shows examples of examining observating files and converting between NSF Astronomy (.ast), Hot load (.hot) and
Comma separated values (.csv) files.   Some prefer viewing/processing data in spreadsheet programs, and using

To make observations in this format, install the West Virginia University Radio Astronomy Instrumentation Laboratory
software available at https://www.github.com/WVURAIL/gr-radio_astro

After running this notebook, run the associated notebook _ConvertObsToAst.ipynb_ to convert back to AST files.

In [1]:
# To simpify working with observations in different directories the "rasnames"
# package will search for .ast and .hot files in lists of directories, as well as a list of
# file names
import rasnames

In [2]:
# look at all files in the data directory
fullnames, nfull = rasnames.splitNames("data", doDebug=False)

In [3]:
# get last file name
astname = fullnames[nfull-1]
if nfull < 1:
        print("Problem finding files: N=%d" % (nfull))
        print("Current Directory: ")
        !pwd
# now show the file name
print("Last of % d files is: %s" % (nfull, astname))

Last of  44 files is: data/20-11-28T195828.ast


In [4]:
# get first file name, which happens to be a "Hot" file
hotname = fullnames[0]
# now show the file name
print("First of % d files is: %s" % (nfull, hotname))

First of  44 files is: data/20-11-28T180022.hot


### Files above show: 1) Sample/Spectral channel number, 2) Frequency (Hz), 3) Intensity (Counts)
We want to know the system temperature of this telescope.   We assume the earth (ie ground) has a temperature of 290 Kelvin
We assume the empty sky has a temperature of 5 Kelvin from the Cosmic Microwave Background and a little bit of Atmosphere

In [5]:
Thot = 290
Tcold = 5

In [6]:
# get one set of hot values
hotvalues = ! head -1024 {hotname} | tail -1
print(hotvalues)

['0964 1421767578 997.8760']


In [7]:
# get one set of cold values
astvalues = ! head -1024 {astname} | tail -1
print(astvalues)

['0964 1421767578 432.7318']


 ### Now that hot and cold intensities are available, Calculate the system temperature for one channel

In [8]:
# Select out the hot and cold intensities from the files
hotstr = hotvalues[0]
hotparts = hotstr.split()
#print(hotparts)
Nhot = float(hotparts[2])
aststr = astvalues[0]
astparts = aststr.split()
#print(astparts)  # diagnosis not needed now
Ncold = float(astparts[2])

# The calibration is done using the "Y-value" method.
Where Y: ratio of hot-load-counts over cold-sky-counts.  
This Y ratio is independent of the gain of the system.

This method is documented on the web.  Ie for example see:
https://www.testworld.com/wp-content/uploads/noise-figure-measurement-accuracy-the-y-factor-method.pdf


In [10]:
# The calibration is done using the "Y-value" system
Y = Nhot/Ncold
Tsys = (Thot - (Y * Tcold)) / (Y - 1.)
print("Y: %7.3f => Tsys = %10.3f K" % (Y, Tsys))

Y:   2.306 => Tsys =    213.225 K


## You've found the Sensitivity of this telescope is good enough for great observations of the Milky Way!
For the example observation, the system temperature is roughly 200 Kelvins, which
is sufficiently low that the Milky Way is easily detected by telescopes.

 ### Now load observing file conversion programs

In [11]:
# Now import the conversion routines
import convertobs

In [12]:
newnames, nnew = convertobs.toCsv(fullnames)
print("New files created: %d; %s to %s" % (nnew, newnames[0], newnames[nnew-1]))

New files created: 44; data/20-11-28T180022.csv to data/20-11-28T195828.csv


In [13]:
# now look at the file header
!head -10 {newnames[0]}

#,        , File: 20-11-28T180022.hot
#,NOTEA   , RSP1A+GPIOLabs Pi #1 On Horn
#,NOTEB   , Sdrplay 8 MHz Test
#,OBSERVER, Science Aficionado
#,DEVICE  , SDRPlay RSP1A
#,DATADIR , ../data/
#,SITE    , RSP1A+GPIOLabs Pi #1 On Horn
#,CITY    , Green Bank
#,REGION  , West Virginia
#,COUNTRY , US


In [14]:
# now look at the data 
astvalues = ! head -1024 {newnames[0]} | tail -1
print(astvalues)

['0964,  1421767578,  997.8760']


## You've now examined the raw observing files for a Horn Radio Telescope and converted Astronomy format (.ast and .hot) data into Comma-Separted-Values (.csv) ASCII format for processing inside of spreadsheets.
## You've also found the sensitivity of this horn radio telescope is good enough for observations of the Milky Way.



In [15]:
# select a new directory name
outdir = "converted"
# use the shell command to create the directory
import os
if os.path.isdir(outdir):
    print("Directory '%s' already exists" % (outdir))
else:
    print("Creating directory '%s'" % (outdir))
    !mkdir {outdir}
# now for all converted files
for iii in range(len(newnames)):
    aname = newnames[iii]
    # if file has not already been moved
    if os.path.isfile(aname):
        ! mv {aname} {outdir}
# end for all files converted

Creating directory 'converted'


## You've converted observing files to .csv format and moved them to a new directory you created.
# Congratulations!